<a href="https://colab.research.google.com/github/Adithyan773/IKEA_recomendation_system/blob/main/ikea_resnet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow

In [ ]:
!pip install -q faiss-cpu

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
import pandas as pd
import os
import faiss

df = pd.read_csv('/content/ikea_data_img_fixed.csv')
image_dir = '/content/drive/MyDrive/images/images'
image_paths = [os.path.join(image_dir, f"{row['item_id']}.jpg") for _, row in df.iterrows() if pd.notna(row['item_id'])]
valid_paths = [path for path in image_paths if os.path.exists(path)]

base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
image_input = Input(shape=(224, 224, 3))
x = base_model(image_input, training=False)
image_embedding = Dense(256, activation='relu', name='image_embedding')(x)
image_model = Model(inputs=image_input, outputs=image_embedding)

def preprocess_image(image_path):
    try:
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        return preprocess_input(img_array)
    except:
        return np.zeros((1, 224, 224, 3))

image_data = np.vstack([preprocess_image(path) for path in valid_paths])

image_embeddings = image_model.predict(image_data)

d = 256
image_index = faiss.IndexFlatL2(d)
image_index.add(image_embeddings)

faiss.write_index(image_index, '/content/image_embeddings_fixed.faiss')
np.save('/content/image_paths_fixed.npy', np.array(valid_paths))

print("Image embedings shape ", image_embeddings.shape)